In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [2]:
import re    # for regular expressions
from string import punctuation
from nltk.stem import SnowballStemmer    #if you are brave enough to do stemming
from nltk.corpus import stopwords      #if you want to remove stopwords
from nltk.tokenize import word_tokenize

In [3]:
import string

In [4]:
products = pd.read_csv('products_train.csv')
products.head()

,id,locale,title,price,brand,color,size,model,material,author,desc
0,B005ZSSN10,DE,RED DRAGON Amberjack 3 - Steel Tip 22 Gramm Wo...,30.95,RED DRAGON,NaN,NaN,RDD0089,NaN,NaN,Amberjacks Steel Dartpfeile sind verfügbar in ...
1,B08PRYN6LD,DE,Simply Keto Lower Carb* Schokodrops ohne Zucke...,17.90,Simply Keto,NaN,750 g (1er Pack),NaN,NaN,NaN,🌱 NATÜRLICHE SÜSSE DURCH ERYTHRIT - Wir stelle...
2,B09MBZJ48V,DE,"Sennheiser 508377 PC 5.2 Chat, Stilvolles Mult...",68.89,Sennheiser,Multi-Colour,One size,508377,Kunstleder,NaN,3.5 MM BUCHSE - Kann problemlos an Geräte mit ...
3,B08ZN6F26S,DE,AmyBenton Auto ab 1 2 3 ahre - Baby Aufziehbar...,18.99,Amy & Benton,Animal Car,NaN,2008B,aufziehauto 1 jahr,NaN,【Auto aufziehbar】: Drücken Sie einfach leicht ...
4,B094DGRV7D,DE,PLAYMOBIL - 70522 - Cavaliere mit grauem Pony,7.17,PLAYMOBIL,Nicht Zutreffend.,OneSize,70522,Polypropylen,NaN,Inhalt: 1 Stück


In [5]:
products = products.dropna(subset=['title'])

In [6]:
products['locale'].unique()

array(['DE', 'JP', 'UK', 'ES', 'FR', 'IT'], dtype=object)

In [7]:
products_uk = products[products['locale']=='UK']

In [8]:
products_uk.head()

,id,locale,title,price,brand,color,size,model,material,author,desc
913336,B087LZNPHS,UK,"SOCHOW Sherpa Fleece Throw Blanket, Double-Sid...",24.99,SOCHOW,Teal Green,127cm×150cm,NaN,100% Polyester,NaN,COLOR: The sherpa throw blanket is available i...
913337,B08THFN1KX,UK,Hippowarehouse Personalised Photo Printed Mous...,9.95,Hippowarehouse,White,240mm x 190mm x 60mm,50245-Mat-Perso,Rubber,NaN,Competitively priced
913338,0804185328,UK,"500 Easy Recipes for Every Machine, Both Stove...",16.49,Clarkson Potter,White,NaN,NaN,NaN,"Scarbrough, Mark",NaN
913339,B09VBKDBW6,UK,"TYHJOY Mini Bag Sealer, Handheld Vacuum Heat S...",11.99,TYHJOY,Black,NaN,FBA-sealer-black,Acrylonitrile Butadiene Styrene,NaN,【AFTER-SALE】This handheld food heat sealer sho...
913340,B096ZW8B49,UK,Lucosobie Steering Wheel Lock - Car Anti-Theft...,26.99,Lucosobie,Black,NaN,NaN,Alloy Steel,NaN,🔐【 Anti-Friction & Customer First】Each box of ...


In [9]:
products_uk.dtypes

id           object
locale       object
title        object
price       float64
brand        object
color        object
size         object
model        object
material     object
author       object
desc         object
dtype: object

In [10]:
def clean(text, stem_words=True):
    
#     text = re.sub('(?<=[0-9])\,(?=[0-9])', "", text)
#     print(text)
#     text = re.sub('^(?!.[a-zA-Z])(?!.\d)[a-zA-Z0-9]+$', "", text)
    text = re.findall(r'\b(?:[a-zA-Z]+|\d+)\b', text)
    text = " ".join(text)
    return text

In [11]:
products_uk['title'] = products_uk['title'].apply(clean)

/var/folders/lh/0lt7mv811l56hczm0y7dwzym0000gn/T/ipykernel_20177/2897974719.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  products_uk['title'] = products_uk['title'].apply(clean)


In [12]:
np.array(products_uk['title'])

array(['SOCHOW Sherpa Fleece Throw Blanket Double Sided Super Soft Luxurious Plush Blanket Teal Green',
       'Hippowarehouse Personalised Photo Printed Mouse Mat Pad Accessory Black Rubber Base Collage Option Home Office Fathers Day Teacher Present x x',
       '500 Easy Recipes for Every Machine Both Stovetop and Electric A Cookbook',
       ...,
       'Before I Do the new funny and unexpected love story from the author of THIS TIME NEXT YEAR',
       'Black iPhone Charger Cable iPhone Charger Braided Long iPhone Charger Lead USB Fast Charging Cable Compatible with iPhone 11 Pro Xs Max X 8 7 Plus 6 SE iPad and More',
       'Kids B Crafty 100 Mini Pegs Mini Wooden Pegs Mini pegs for photos Small Wooden Pegs Small Pegs Craft Pegs Wooden Pegs Craft'],
      dtype=object)

In [13]:
# load session data
sessions = pd.read_csv('sessions_train.csv')

sessions.head()

,prev_items,next_item,locale
0,['B09W9FND7K' 'B09JSPLN1M'],B09M7GY217,DE
1,['B076THCGSG' 'B007MO8IME' 'B08MF65MLV' 'B001B...,B001B4THSA,DE
2,['B0B1LGXWDS' 'B00AZYORS2' 'B0B1LGXWDS' 'B00AZ...,B0767DTG2Q,DE
3,['B09XMTWDVT' 'B0B4MZZ8MB' 'B0B7HZ2GWX' 'B09XM...,B0B4R9NN4B,DE
4,['B09Y5CSL3T' 'B09Y5DPTXN' 'B09FKD61R8'],B0BGVBKWGZ,DE


In [14]:
# get UK sessions
sessions_uk = sessions[sessions['locale']=='UK']
print(sessions_uk.shape)
sessions_uk.head()


(1182181, 3)


,prev_items,next_item,locale
2090535,['B0BFDL54Y7' 'B0BFDR9X13' 'B07J4WF8VH' 'B07Y2...,B07Y227WNJ,UK
2090536,['B07FM2GLNQ' 'B07GZW3P4W'],B095NNZCR6,UK
2090537,['B0021L95HU' 'B07DDL77RY' 'B07DDL77RY'],B002KA1FZC,UK
2090538,['B0B2WSZYL2' 'B000I8XZ7O'],B000I90TAO,UK
2090539,['B00LW1APOC' 'B00LW1APOC' 'B00OI6NQUI' 'B09HL...,B07H54NZ3K,UK


In [15]:
# substitute ids with titles
import pandas as pd



sessions_uk = sessions_uk.copy()

# Create a dictionary to map product id to title
id_to_title = products_uk.set_index('id')['title'].to_dict()
print(id_to_title.get('B0BFDL54Y7'))


Wokii Vegetable Chopper Vegetable Slicer 14 in 1 Vegetable Chopper Cutter Dicer With Food Container for Onion Veg Garlic Potato Salad Mini Dicer Chopper Cutter


In [16]:
import re
# convert string of ids to list of ids
def convert_ids_string_to_list(ids_string):
    return re.findall(r"'(.*?)'", ids_string)

# convert list of ids to list of titles
sessions_uk['prev_items'] = sessions_uk['prev_items'].apply(convert_ids_string_to_list)

sessions_uk['prev_items'] = sessions_uk['prev_items'].apply(lambda x: [id_to_title.get(i) for i in x if id_to_title.get(i)])


In [17]:


# Replace product IDs in 'next_item' with their corresponding titles.
sessions_uk['next_item'] = sessions_uk['next_item'].apply(lambda x: id_to_title.get(x) if id_to_title.get(x) else x)


In [18]:
# sessions_uk with titles instead of ids. ready for NLP
sessions_uk.head()


,prev_items,next_item,locale
2090535,[Wokii Vegetable Chopper Vegetable Slicer 14 i...,ADOV Vegetable Chopper 14 in 1 Multi Function ...,UK
2090536,[Trust Verto Wireless Ergonomic Mouse Vertical...,Bedsure White Voile Sheer Curtains 2 Panels Wi...,UK
2090537,[Seven Seas Cod Liver Oil Tablets Plus Multivi...,Seven Seas Omega 3 FISH Oil Plus Cod Liver Oil...,UK
2090538,[ABC Products Compatible Replacement Shark 2 i...,Rubie s Official Halloween Haunted House Skele...,UK
2090539,[Command Medium Picture Mounting Strips Pack o...,Command Assorted Picture Hanging Strips White ...,UK


In [19]:
session = sessions_uk['prev_items'].iloc[0]
session 


['Wokii Vegetable Chopper Vegetable Slicer 14 in 1 Vegetable Chopper Cutter Dicer With Food Container for Onion Veg Garlic Potato Salad Mini Dicer Chopper Cutter',
 'Wokii Vegetable Chopper Vegetable Slicer 14 in 1 Vegetable Chopper Cutter Dicer With Food Container for Onion Veg Garlic Potato Salad Mini Dicer Chopper Cutter',
 'Vinsani Kitchen Vegetable Cutter Onion Garlic Fruits Cheese Food Cutter Potato Chopper Salad Slicer with 3 Interchangeable Blade Inserts Set Stainless Steel',
 'ADOV Vegetable Chopper 14 in 1 Multi Function Kitchen Mandoline Slicer 7 Replaceable Stainless Steel Vegetable Cutter with Egg Separator Hand Guard Julienne Grater for Onion Potato Fruits Blue']

In [1]:
import torch
from torch.utils.data import Dataset, DataLoader, random_split
from transformers import T5Tokenizer, T5ForConditionalGeneration, AdamW
from torch.nn.utils.rnn import pad_sequence


tokenizer = T5Tokenizer.from_pretrained('t5-base')

class SessionDataset(Dataset):
    def __init__(self, sessions, tokenizer, max_length=512):
        self.sessions = sessions
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.sessions)

    def __getitem__(self, idx):
        session = self.sessions.iloc[idx]
        encoding = self.tokenizer(session['prev_items'], truncation=True, max_length=self.max_length, padding='max_length')
        target_encoding = self.tokenizer(session['next_item'], truncation=True, max_length=self.max_length, padding='max_length')

        return {
            'input_ids': torch.tensor(encoding.input_ids, dtype=torch.long),
            'attention_mask': torch.tensor(encoding.attention_mask, dtype=torch.long),
            'labels': torch.tensor(target_encoding.input_ids, dtype=torch.long),
        }
def collate_fn(batch):
    input_ids = [item['input_ids'] for item in batch]
    labels = [item['labels'] for item in batch]
    
    input_ids = pad_sequence(input_ids, batch_first=True, padding_value=tokenizer.pad_token_id)
    labels = pad_sequence(labels, batch_first=True, padding_value=-100)
    
    return {'input_ids': input_ids, 'labels': labels}

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


: 

: 

In [1]:




dataset = SessionDataset(sessions_uk, tokenizer)
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True,collate_fn=collate_fn)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False,collate_fn=collate_fn)

model = T5ForConditionalGeneration.from_pretrained('t5-base')

optimizer = AdamW(model.parameters(), lr=1e-4)

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)

for epoch in range(3):  
    running_loss = 0.0
    model.train()  # set model to training mode
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data['input_ids'].to(device), data['labels'].to(device)
        optimizer.zero_grad()
        outputs = model(input_ids=inputs, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        if i % 2000 == 1999: 
            print('[%d, %5d] training loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

    # validation loop
    model.eval()  # set model to eval mode
    running_val_loss = 0.0
    with torch.no_grad():  # no gradients for validation
        for i, data in enumerate(val_loader, 0):
            inputs, labels = data['input_ids'].to(device), data['labels'].to(device)
            outputs = model(input_ids=inputs, labels=labels)
            loss = outputs.loss
            running_val_loss += loss.item()
    print('[%d] validation loss: %.3f' %
          (epoch + 1, running_val_loss / len(val_loader)))

print('Finished Training')

# Save the trained model
torch.save(model.state_dict(), 't5_trained_model.pt')


Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


: 

: 